# C-Python App

In [1]:
from app import app_main

await app_main()

already up to date
hello from counter.py
init counter! counter.count 1
2024-09-02 17:37:55 INFO      new_leaf     app                                   Starting leaf new_leaf
Starting async server on localhost:5000...
2024-09-02 17:37:55 INFO      new_leaf     app                                   loading plugins.core.log with param {}
2024-09-02 17:37:55 INFO      new_leaf     app                                   loading plugins.core.webserver with param {'debug': True}
2024-09-02 17:37:55 INFO      new_leaf     app                                   loading plugins.counter with param {'eid': 'counter.count', 'interval': 1}
so long ...
